# Season_13_data_preparation.ipynb

This is the second version of the software, which was created to support larger amounts of data. Here, the alignment was performed on all of the episodes from the thirteenth season of *Grey's Anatomy*. The input is the following:

 
1.   An XLSX file containing the data from episode 13x01 to 13x24.
2.   A ZIP file containing the subtitles of each episode.


# Libraries

In addition to pandas and re, some additional libraries were used in order to extract and organize the files.

In [17]:
import pandas as pd
import numpy as np
import tempfile
import zipfile
import shutil
import glob
import os
import re

In [18]:
pd.options.mode.chained_assignment = None 

# Importing the files

In [19]:
# Opening season 13 .xlsx file 

excel_path = input('Enter .xlsx file path: ')
df_excel = pd.read_excel(excel_path)

Enter .xlsx file path: /content/coding_season_13.xlsx


In [20]:
# Structure of the .xlsx file

df_excel

,Season,Codice,N_segmento,Inizio,Fine,Durata,PP,PP_rel,SP,SP_rel,MC,MC_rel,Note
0,GAS13,GAS13E01,1,00:00:00,00:00:44,00:00:44,NaN,NaN,NaN,NaN,NaN,NaN,PREVIOUSLY
1,GAS13,GAS13E01,2,00:00:44,00:00:49,00:00:05,NaN,NaN,NaN,NaN,NaN,NaN,Contestualizzazione città
2,GAS13,GAS13E01,3,00:00:49,00:02:18,00:01:29,0,NaN,6,"SP3,SP4,SP5,SP6",0,NaN,"Pg+monologo, Inizio montaggio alternato"
3,GAS13,GAS13E01,4,00:02:18,00:02:36,00:00:18,2,PP1,2,SP3,2,MC1,NaN
4,GAS13,GAS13E01,5,00:02:36,00:03:18,00:00:42,0,NaN,6,SP7,0,NaN,Fine montaggio alternato
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575,GAS13,GAS13E24,NaN,00:41:23.130000,00:41:42.590000,00:00:19.460000,NaN,NaN,NaN,NaN,NaN,NaN,Contestualizzazione
1576,GAS13,GAS13E24,NaN,00:41:42.590000,00:41:47.410000,00:00:04.820000,0,NaN,0,NaN,6,SP5,NaN
1577,GAS13,GAS13E24,NaN,00:41:47.410000,00:41:59.060000,00:00:11.650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1578,GAS13,GAS13E24,NaN,00:41:59.060000,00:42:09.570000,00:00:10.510000,0,NaN,0,NaN,6,SP10,NaN


In [21]:
# Unzipping the contents of the .zip file to a temporary directory

temp_dir = tempfile.TemporaryDirectory()
zip_path = input('Enter .zip file path: ')

with zipfile.ZipFile(zip_path) as z:
  
   for zip_info in z.infolist():
    if zip_info.filename[-1] == '/':
      continue

    zip_info.filename = os.path.basename(zip_info.filename)
    z.extract(zip_info, temp_dir.name)

print('Temp directory path: ' + str(temp_dir.name))

Enter .zip file path: /content/subtitles_season_13.zip
Temp directory path: /tmp/tmpdd53eas3


In [22]:
# Getting the paths of the extracted .srt files

srt_paths = []

for filename in sorted(glob.glob(os.path.join(temp_dir.name, '*.srt'))):
  srt_paths.append(filename)

# Cleaning the data

Before aligning the files, the following cleaning was performed on the Excel file:
*   Timestamps containing milliseconds were rounded to the nearest second
*   Typographical errors were fixed (e.g. "NA ", "000:09:20,487")
*   NaNs were replaced with 0s
*   Floats were converted to integers





In [23]:
# Dropping columns

df_excel = df_excel.drop(['Season', 'N_segmento', 'Durata', 'PP_rel', 'SP_rel', 'MC_rel', 'Note'], axis=1)

In [24]:
# Rounding start timestamps

df_excel['Inizio'] = df_excel['Inizio'].astype(str)
df_excel['Inizio'] = pd.to_datetime(df_excel['Inizio'], errors = 'coerce')
df_excel['Inizio'] = df_excel['Inizio'].dt.round('1s')
df_excel['Inizio'] = df_excel['Inizio'].dt.time

In [25]:
# Rounding end timestamps

df_excel['Fine'] = df_excel['Fine'].astype(str)
df_excel['Fine'] = df_excel['Fine'].apply(lambda x : '00:09:20' if x=='000:09:20,487' else x) # Fixing a value in line 614
df_excel['Fine'] = pd.to_datetime(df_excel['Fine'], errors = 'coerce')
df_excel['Fine'] = df_excel['Fine'].dt.round('1s')
df_excel['Fine'] = df_excel['Fine'].dt.time

In [26]:
# Fixing other typographical errors

df_excel.PP.replace('NA ', np.nan, inplace=True)
df_excel.SP.replace('NA ', np.nan, inplace=True)
df_excel.MC.replace('NA ', np.nan, inplace=True)
df_excel.MC.replace('SP10', np.nan, inplace=True) # Fixing a value in line 786

In [27]:
# Replacing NaNs with 0s

df_excel = df_excel.fillna(0)

In [28]:
# Converting floats to ints

df_excel['PP'] = df_excel['PP'].astype(int)
df_excel['SP'] = df_excel['SP'].astype(int)
df_excel['MC'] = df_excel['MC'].astype(int)

In [29]:
# Structure of the .xlsx file after cleaning

df_excel

,Codice,Inizio,Fine,PP,SP,MC
0,GAS13E01,00:00:00,00:00:44,0,0,0
1,GAS13E01,00:00:44,00:00:49,0,0,0
2,GAS13E01,00:00:49,00:02:18,0,6,0
3,GAS13E01,00:02:18,00:02:36,2,2,2
4,GAS13E01,00:02:36,00:03:18,0,6,0
...,...,...,...,...,...,...
1575,GAS13E24,00:41:23,00:41:43,0,0,0
1576,GAS13E24,00:41:43,00:41:47,0,0,6
1577,GAS13E24,00:41:47,00:41:59,0,0,0
1578,GAS13E24,00:41:59,00:42:10,0,0,6


# Splitting the Excel file by episode

The idea behind the following section is to split the Excel file so that each episode is stored as a separate DataFrame. This way, the procedure followed in episode_12x01_data_preparation.ipynb can be reimplemented for multiple episodes as well.

In [30]:
# Finding where each episode starts and ends

episode_start = []
episode_end = []
episode_start.append(0)

for idx, val in enumerate(df_excel.Codice):

    try:
      if df_excel.Codice[idx] != df_excel.Codice[idx+1]:
        episode_end.append(idx)
        episode_start.append(idx+1)
    
    except Exception:
      episode_end.append(len(df_excel.Codice))

In [31]:
# Boundaries of each episode

print(episode_start)
print(episode_end)

[0, 47, 112, 162, 221, 278, 331, 387, 443, 497, 542, 601, 654, 714, 818, 893, 963, 1034, 1098, 1171, 1261, 1324, 1392, 1474]
[46, 111, 161, 220, 277, 330, 386, 442, 496, 541, 600, 653, 713, 817, 892, 962, 1033, 1097, 1170, 1260, 1323, 1391, 1473, 1580]


In [32]:
# Storing each episode as a separate DataFrame

df_episodes_list = []

for ep_start, ep_end in zip(episode_start, episode_end):

  df_episode = df_excel.loc[ep_start:ep_end]
  df_episodes_list.append(df_episode)

print('Total episodes: ' + str(len(df_episodes_list)))

Total episodes: 24


# Defining the functions

The following functions reimplement the procedure that was described in episode_12x01_data_preparation.ipynb. 

In [33]:
# Computing the mean of each subtitle's timespan, which is used to align the subs with the Excel segments

def average_time(average_times, start_times_datetime, end_times_datetime):

  for s, e in zip(start_times_datetime, end_times_datetime):
    ts1 = s
    ts2 = e
    average_time = ts1+(ts2-ts1)/2
    average_times.append(average_time)

The data extracted from the subtitles and the Excel file can be passed to the functions align_segments() or align_subs(). When executing the code, the user is asked to choose whether to align the data by segment (=to group subtitles that are part of the same segment) or by subtitle (=to display each subtitle independently).

In [34]:
# If the user selects option [1] when alignment_type() is called, the data is aligned by segment

def align_segments(df_subs, codice, inizio, fine, pp, sp, mc, aligned_subs):

  for code, start, end, pp, sp, mc in zip(codice, inizio, fine, pp, sp, mc):
    
    mask = (df_subs['Average'] > start) & (df_subs['Average'] <= end)
    mask_df = df_subs.loc[mask]

    mask_df = mask_df.drop(['Average'], axis=1)
    mask_df['PP'] = pp
    mask_df = mask_df.groupby('PP')
    mask_df = mask_df['Subtitle text'].agg(lambda column: ''.join(column))
    mask_df = mask_df.reset_index(name='Subtitle text')
    mask_df.rename(columns={'Subtitle text':'Segment text'}, inplace=True)
    mask_df['SP'] = sp
    mask_df['MC'] = mc
    mask_df['Segment start'] = start   
    mask_df['Segment end'] = end
    mask_df['Code'] = code
    mask_df = mask_df[['Code', 'Segment start', 'Segment end', 'PP', 'SP', 'MC', 'Segment text']]
    aligned_subs.append(mask_df)

In [35]:
# If the user selects option [2] when alignment_type() is called, the data is aligned by subtitle

def align_subs(df_subs, codice, inizio, fine, pp, sp, mc, aligned_subs):

  for code, start, end, pp, sp, mc in zip(codice, inizio, fine, pp, sp, mc):
    
    mask = (df_subs['Average'] > start) & (df_subs['Average'] <= end)
    mask_df = df_subs.loc[mask]

    mask_df['PP'] = pp
    mask_df['SP'] = sp
    mask_df['MC'] = mc
    mask_df['Subtitle start'] = mask_df['Start'].astype(str)
    mask_df['Subtitle start'] = mask_df['Subtitle start'].str.replace(r'000$', '', regex=True)  
    mask_df['Subtitle end'] = mask_df['End'].astype(str)
    mask_df['Subtitle end'] = mask_df['Subtitle end'].str.replace(r'000$', '', regex=True) 
    mask_df['Start'] = start   
    mask_df['End'] = end
    mask_df['Code'] = code
    mask_df = mask_df[['Code', 'Subtitle start', 'Subtitle end', 'PP', 'SP', 'MC', 'Subtitle text']]
    aligned_subs.append(mask_df)

For each episode, the process is the same as before:

1.   Parsing and storing subtitles in a DataFrame
2.   Calculating the average of the subtitles' timespans
3.   Using the average to perform the alignment either by segment or by subtitle
4.   Storing the episode's DataFrame in a list 
5.   Repeating the process until all of the episodes have been aligned

In [36]:
def process_files(xlsx_file, srt_file, reply, aligned_episodes):

  df_excel = xlsx_file

  with open(srt_file, 'r') as f:
    subs = f.read().splitlines()

    # Parsing .srt file

    re_pattern = r'[0-9]{2}:[0-9]{2}:[0-9]{2},[0-9]{3} -->'
    regex = re.compile(re_pattern)

    start_times = list(filter(regex.search, subs))
    start_times = [time.split(' ')[0] for time in start_times]
    start_times_datetime1 = pd.to_datetime(start_times, format='%H:%M:%S,%f')
    start_times_datetime2 = pd.Series(start_times_datetime1, name='Start').dt.time

    end_times = list(filter(regex.search,subs))
    end_times = [time.split(' ')[2] for time in end_times]
    end_times_datetime1 = pd.to_datetime(end_times, format='%H:%M:%S,%f')
    end_times_datetime2 = pd.Series(end_times_datetime1, name='End').dt.time

    subtitles = [[]]

    for sub in subs:
        if re.match(re_pattern, sub):
          subtitles[-1].pop()
          subtitles.append([])
        else:
          subtitles[-1].append(sub)

    subtitles = subtitles[1:]
    subtitles = [' '.join(x) for x in subtitles]
    subtitles = pd.Series(subtitles, name='Subtitle text')

    # Averaging timespans

    average_times = []
    average_time(average_times, start_times_datetime1, end_times_datetime1)
    average_times = pd.Series(average_times, name='Average').dt.time

    # Creating .srt DataFrame

    df_subs = pd.concat([start_times_datetime2, end_times_datetime2, average_times, subtitles], axis=1)
    df_subs = df_subs.reset_index(drop=True)

    # Matching subtitles with segments

    aligned_subs = []

    if reply == '1':
      align_segments(df_subs, df_excel.Codice, df_excel.Inizio, df_excel.Fine, df_excel.PP, df_excel.SP, df_excel.MC, aligned_subs)
    
    if reply == '2':
      align_subs(df_subs, df_excel.Codice, df_excel.Inizio, df_excel.Fine, df_excel.PP, df_excel.SP, df_excel.MC, aligned_subs)

    else:
      pass

    # Appending aligned episode to episode list

    aligned_file = pd.concat(aligned_subs)
    aligned_file.reset_index(drop=True)
    aligned_episodes.append(aligned_file)

# Aligning the episodes

Calling the functions described above to perform the alignment.

In [37]:
aligned_episodes = []

def alignment_type(question):

  reply = str(input(question)).lower().strip()
  
  print('Aligning episodes...')

  for xlsx, srt in zip(df_episodes_list, srt_paths):
      process_files(xlsx, srt, reply, aligned_episodes)

In [38]:
alignment_type('Align by segment [1] or Align by subtitle [2]? ') 

Align by segment [1] or Align by subtitle [2]? 1
Aligning episodes...


The list containing each episode's DataFrame is merged with pd.concat() to create a single DataFrame with all of the aligned data.

In [43]:
season_df = pd.concat(aligned_episodes)
season_df = season_df.reset_index(drop=True)
season_df

,Code,Segment start,Segment end,PP,SP,MC,Segment text
0,GAS13E01,00:00:00,00:00:44,0,0,0,"<i>Previously on ""Grey's Anatomy""...</i> I wan..."
1,GAS13E01,00:00:44,00:00:49,0,0,0,♪
2,GAS13E01,00:00:49,00:02:18,0,6,0,♪ I ain't got no problem ♪ ♪ That's for real ♪...
3,GAS13E01,00:02:18,00:02:36,2,2,2,[Siren wails] Isaac: What do we got? We got a ...
4,GAS13E01,00:02:36,00:03:18,0,6,0,Two champagnes. You got it. I thought you were...
...,...,...,...,...,...,...,...
1466,GAS13E24,00:40:58,00:41:23,0,0,6,"[Engine starts] <i>Nobody wakes up thinking, ""..."
1467,GAS13E24,00:41:23,00:41:43,0,0,0,"<i>Sometimes, we wake up, we face our fears......"
1468,GAS13E24,00:41:43,00:41:47,0,0,6,<i>We take them by the hand.</i> ♪♪
1469,GAS13E24,00:41:59,00:42:10,0,0,6,"♪ So far away ♪ <i>- And we stand there, waiti..."


# Deleting temporary files

Deleting the temporary directory where the extracted .srt files were stored.

In [40]:
shutil.rmtree(temp_dir.name)

# Exporting to Excel

In [41]:
def yes_or_no(question):
    reply = str(input(question+' [y/n]: ')).lower().strip()
    if reply[0] == 'y':
        return season_df.to_excel('season_13_with_subtitles.xlsx')
        print('Exported to Excel')
    if reply[0] == 'n':
        pass
    else:
        pass

In [44]:
yes_or_no('Export to Excel?') 

Export to Excel? [y/n]: y
